<a href="https://colab.research.google.com/github/nlodder/ENPH353_Lab2/blob/main/ENPH353_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import userdata
import os

# Retrieve credentials
token = userdata.get('GH_TOKEN')
username = "nlodder"
repo = "ENPH353_Lab2"

# Clone the repo (using token for Write access)
!git clone https://{token}@github.com/{username}/{repo}.git

# Move into the directory
%cd {repo}

# Set Git Identity
!git config --global user.email "nathan.lodder@proton.me"
!git config --global user.name "Nathan Lodder"

Cloning into 'ENPH353_Lab2'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 17 (delta 2), reused 6 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 8.48 MiB | 10.92 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/ENPH353_Lab2/ENPH353_Lab2


In [7]:
import cv2 as cv
import numpy as np

# --- CONFIGURATION ---
VIDEO_PATH = "/content/ENPH353_Lab2/raw_video_feed.mp4"
DOT_RADIUS = 20
DOT_COLOR = (0, 0, 255)
DARKNESS_THRESHOLD = 20

def preprocess_slice(bgr_slice):
    """
    @brief Converts BGR to HSV and returns the V channel average.
    @param bgr_slice raw slice from video file in BGR
    """
    hsv = cv.cvtColor(bgr_slice, cv.COLOR_BGR2HSV)
    avg_v = np.mean(hsv[:, :, 2])
    return hsv, avg_v

def get_line_mask(hsv_slice, avg_v):
    """
    @brief Creates the binary mask based on adaptive brightness
    @param hsv_slice slice in HSV color space
    @param avg_v average brightness of the slice
    """
    upper_v = int(np.clip(avg_v - DARKNESS_THRESHOLD, 0, 255))
    lower_bound = np.array([0, 0, 0], dtype=np.uint8)
    upper_bound = np.array([179, 255, upper_v], dtype=np.uint8)
    return cv.inRange(hsv_slice, lower_bound, upper_bound)

def find_cent_x_from_mask(mask):
    """
    @brief Returns the horizontal center of the line from the mask.
    @param mask binary mask of the slice
    """
    moment = cv.moments(mask)
    # Check if the mask actually has white pixels to avoid DivisionByZero
    if moment["m00"] > 0:
        cent_horiz = int(moment["m10"] / moment["m00"])
        return cent_horiz
    else:
        return None

def main():
    cap = cv.VideoCapture(VIDEO_PATH)
    cap_fps = cap.get(cv.CAP_PROP_FPS)
    (cap_width, cap_height) = (int(cap.get(cv.CAP_PROP_FRAME_WIDTH)),
                               int(cap.get(cv.CAP_PROP_FRAME_HEIGHT)))
    cent_y = cap_height - DOT_RADIUS - 1
    write_obj  = cv.VideoWriter(
        "labeled_video_feed.mp4", cv.VideoWriter_fourcc(*'mp4v'), cap_fps, (cap_width, cap_height))
    filter_write_obj = cv.VideoWriter(
        "filter_vid.mp4", cv.VideoWriter_fourcc(*'mp4v'), cap_fps, (cap_width, 2*DOT_RADIUS))

    last_x = int(cap.get(cv.CAP_PROP_FRAME_WIDTH) / 2)

    while True:
        ret, frame = cap.read()
        if not ret: break

        # 1. Isolate Logic
        y_start = cap_height - 2 * DOT_RADIUS - 1
        roi = frame[y_start : cap_height, :]

        hsv_roi, avg_v = preprocess_slice(roi)
        mask = get_line_mask(hsv_roi, avg_v)

        # 2. Update state
        current_x = find_cent_x_from_mask(mask)
        if current_x is not None:
            last_x = current_x

        # 3. Output
        cv.circle(frame, (last_x, cent_y), DOT_RADIUS, DOT_COLOR, -1)
        write_obj.write(frame)
        filter_write_obj.write(cv.cvtColor(mask, cv.COLOR_GRAY2BGR))

    cap.release()
    write_obj.release()
    filter_write_obj.release()

if __name__ == "__main__":
    main()